# Import

### File

In [1]:
import os
# import csv

### Scientific

In [2]:
import numpy as np
import pandas as pd

### Data Structure

In [3]:
from collections import Counter
from collections import defaultdict

### Database

In [4]:
import sqlite3
from sqlite3 import Error

### Functions

In [5]:
# from itertools import permutations

### Linear Algebra

In [6]:
# import scipy.linalg as la

### Date and Time

In [7]:
import pytz
from datetime import datetime
from datetime import timedelta

### Machine Learning

In [8]:
# import tensorflow as tf
# from sklearn.preprocessing import QuantileTransformer

### Network

In [9]:
import networkx as nx
# from networkx.algorithms import bipartite

### Plot

In [10]:
# from pylab import *
import matplotlib as mpl
import matplotlib.pyplot as plt

Use `inline` for static and `notebook` for interactive

In [11]:
%matplotlib inline
# %matplotlib notebook

Change configuration of the default plotting

In [12]:
# (1) simple

# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

In [13]:
# (2) pretty

mpl.rc('xtick', labelsize=14, color="#222222")  # 222222 is dark grey
mpl.rc('ytick', labelsize=14, color="#222222")

mpl.rc('font', **{'family': 'sans-serif', 'sans-serif': ['Arial']})
mpl.rc('font', size=16)

mpl.rc('xtick.major', size=6, width=1)
mpl.rc('xtick.minor', size=3, width=1)
mpl.rc('ytick.major', size=6, width=1)
mpl.rc('ytick.minor', size=3, width=1)

mpl.rc('axes', linewidth=1, edgecolor="#222222", labelcolor="#222222")
mpl.rc('text', usetex=False, color="#222222")

# Functions

In [14]:
PROJECT_ROOT_DIR = '.'
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)


def fig_save(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    """
    Function for saving figures
    """
    path = os.path.join(IMAGES_PATH, fig_id + '.' + fig_extension)
    print('saving figure ...', fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


def dir_walk(path, ext='', save=False):
    """
    Walk through a directory, find all the file with specified extension
    """
    # set ext to a specific extension if needed
    f = []
    for root, dirs, files in os.walk(path):
        for file in files:
            # relative path of file
            relative_path = os.path.join(root, file)
            # extention (type of file)
            ext_of_file = os.path.splitext(relative_path)[-1].lower()[1:]
            # if extension is set and equal to what we want
            if ext != '' and ext_of_file == ext:
                f.append(os.path.abspath(relative_path))
            # if extension is not set add the file anyway
            else:
                f.append(os.path.abspath(relative_path))
    f.sort()
    if save: np.savetxt('files.csv', f, delimiter=',', fmt='%s')
    return f


def colors_create(number_of_colors=1, color_map='Wistia', output=False):
    """
    create a series of colors from the selected spectrum, e.g., Wistia (cold to hot)
    """
    color_list = []
    cmap = cm.get_cmap(color_map, number_of_colors)
    for i in range(cmap.N):
        rgb = cmap(
            i)[:3]  # will return rgba, we take only first 3 so we get rgb
        # print(matplotlib.colors.rgb2hex(rgb))
        color_list.append(matplotlib.colors.rgb2hex(rgb))
    if output:
        for i in range(len(color_list)):
            plt.scatter(i, 1, c=color_list[i], s=20)
    return color_list


def dict_add(dictionary, key, value):
    """
    Add a key:value to dictionary if key does not already exist
    """
    if key not in dictionary:
        dictionary[key] = value


def dict_lookup(dictionary, key):
    """
    Search the given KEY in dictionary ...
    found -> return its value (could be an index assign to that value)
    not found -> add the key and return its value (which is a new index)
    useful for creating hash table of KEY->INDEX
    """
    value = 0
    if key not in dictionary:
        value = len(dictionary)
        dictionary[key] = value
    else:
        value = dictionary.get(key)
    return value

# Database

In [15]:
DB_PATH = os.path.join(PROJECT_ROOT_DIR, 'data', 'yonsei.db')

DB_PATHS = [
    os.path.join(PROJECT_ROOT_DIR, 'data', 'LifeMap_GS' + str(x) + '.db')
    for x in range(1, 13)
]


def db_connect(db):
    """
    Connect to a database having its file name
    """
    try:
        conn = sqlite3.connect(db)
        return conn
    except Error as e:
        print(e)
    return None


def db_close(conn):
    """
    Close the connection to a database
    """
    conn.close()


def db_row_count(conn, table_name, print_out=False):
    """
    Calculate the number of entries in a table of a database having its file name
    """
    cur = conn.cursor()
    cur.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cur.fetchall()
    if print_out:
        print('\n|DB|={}'.format(count[0][0]))
    return count[0][0]


def db_create(db, query):
    """
    Create a database based on the input CREATE query
    """
    conn = db_connect(db)
    try:
        cur = conn.cursor()
        cur.execute(query)
    except Error as e:
        print(e)
    conn.close()


def db_insert_many(db, query, data):
    """
    Insert many (more than one) row to a table of a database
    """
    conn = db_connect(db)
    try:
        cur = conn.cursor()
        cur.executemany(query, data)
        conn.commit()
    except Error as e:
        print(e)
    conn.close()

# Time Zones

In [16]:
old_timezone = pytz.timezone('UTC')
new_timezone = pytz.timezone('Asia/Seoul')

# Data Processing

Read one of the dataset (e.g. user-1 -> LifeMap_GS1.db) and then read the entries

In [18]:
print(DB_PATHS[0])

conn = db_connect(DB_PATHS[0])
query = f"""
SELECT *
FROM apTable
ORDER BY _time_ap
LIMIT 100
"""
df = pd.read_sql_query(query, conn)
db_close(conn)

./data/LifeMap_GS1.db


In [19]:
df.head()

,_ap_id,_node_id,_bssid,_ssid,_open,_signal,_signal_deviation,_sample_count,_time_ap
0,13,1,00:kg:9f:6e:wr:1e,turbulence,[WPA-PSK-TKIP],-81.0,0.0,2,20110308113017TUE
1,16,1,00:bl:bb:79:2k:8b,李쏀썕 �댁쓽 �ㅽ듃�뚰겕,[WPA2-PSK-CCMP],-82.0,0.0,2,20110308113017TUE
2,17,1,00:6w:3f:53:rp:3a,csp,[WPA-PSK-TKIP],-84.0,0.0,2,20110308113017TUE
3,25,1,00:bl:66:1e:hh:58,Env-lab,[WEP],-82.0,0.0,2,20110308113017TUE
4,26,1,00:kg:9f:94:da:bc,IDLab,[WEP],-85.0,0.0,2,20110308113017TUE


Read all users dataset and create a single DB containing WIFI logs in the logs table

In [0]:
users = {}  # list of users
ssids = {}  # list of SSIDs
bssids = {}  # list of BSSIDs
locations = {}  # list of location or {(lat,long):id}

user_column = []
ssid_column = []
bssid_column = []
location_column = []
latitude_column = []
longitude_column = []
accuracy_column = []

_ssid = []
_bssid = []
_time_ap = []
_signal = []
_signal_deviation = []
_sample_count = []
_open = []

for db_file in DB_PATHS:
    print(db_file)

    user_id = int(db_file.split('/')[-1].split('.')[0][10:])
    dict_lookup(users, user_id)  # add user_id to dictionary, if not exist

    query = f"""
    SELECT _node_id,_ssid,_bssid,_time_ap,_signal,_signal_deviation,_sample_count,_open
    FROM apTable
    ORDER BY _time_ap,_ssid,_bssid
    """

    conn = db_connect(db_file)
    df_iterator = pd.read_sql_query(query, conn, chunksize=1000)

    for df in df_iterator:
        # update original columns from DB
        _ssid.extend(list(df['_ssid']))
        _bssid.extend(list(df['_bssid']))
        _time_ap.extend(list(df['_time_ap']))
        _signal.extend(list(df['_signal']))
        _signal_deviation.extend(list(df['_signal_deviation']))
        _sample_count.extend(list(df['_sample_count']))
        _open.extend(list(df['_open']))

        # update/create user_column
        user_column.extend([user_id] * len(df))

        # update/create ssid_column
        for ssid in df['_ssid']:
            ssid_column.append(dict_lookup(ssids, ssid))

        # update/create bssid_column
        for bssid in df['_bssid']:
            bssid_column.append(dict_lookup(bssids, bssid))

        # update latitude, longitude, accuracy, and location columns
        for location in df['_node_id']:
            # to fix empty df error
#             last_record = pd.Series([])
#             query_new = f"""
#             SELECT _latitude,_longitude,_accuracy
#             FROM locationTable
#             WHERE _node_id = {location}
#             """
#             r = pd.read_sql_query(query_new, conn)
#             if len(r) > 0:
#                 last_record = r
#             else:
#                 print(r)
#                 print(last_record)
#                 print(user_id, location)
#                 continue
#             result = last_record.iloc[0]

            query_new = f"""
            SELECT _latitude,_longitude,_accuracy
            FROM locationTable
            WHERE _node_id = {location}
            """
            result = pd.read_sql_query(query_new, conn).iloc[0]

            latitude_column.append(result['_latitude'])
            longitude_column.append(result['_longitude'])
            accuracy_column.append(result['_accuracy'])
            location_column.append(
                dict_lookup(locations, (result['_latitude'], result['_longitude'])))

    db_close(conn)

data = pd.DataFrame({
    'user': user_column,
    'ssid': ssid_column,
    'bssid': bssid_column,
    'location': location_column,
    'latitude': latitude_column,
    'longitude': longitude_column,
    'accuracy': accuracy_column,
    'time': _time_ap,
    'ssid_name': _ssid,
    'bssid_name': _bssid,
    'signal': _signal,
    'deviation': _signal_deviation,
    'sample_count': _sample_count,
    'type': _open
})

/home/alift/cloud/yonsei/data/LifeMap_GS1.db
/home/alift/cloud/yonsei/data/LifeMap_GS10.db
/home/alift/cloud/yonsei/data/LifeMap_GS11.db
/home/alift/cloud/yonsei/data/LifeMap_GS12.db
/home/alift/cloud/yonsei/data/LifeMap_GS2.db
/home/alift/cloud/yonsei/data/LifeMap_GS3.db
/home/alift/cloud/yonsei/data/LifeMap_GS4.db
/home/alift/cloud/yonsei/data/LifeMap_GS5.db
/home/alift/cloud/yonsei/data/LifeMap_GS6.db
/home/alift/cloud/yonsei/data/LifeMap_GS7.db
/home/alift/cloud/yonsei/data/LifeMap_GS8.db
/home/alift/cloud/yonsei/data/LifeMap_GS9.db


In [0]:
data.head()

,user,ssid,bssid,location,latitude,longitude,accuracy,time,ssid_name,bssid_name,signal,deviation,sample_count,type
0,1,0,0,0,37561812,126935396,60,20110308113017TUE,Env-lab,00:bl:66:1e:hh:58,-82.000000,0.000000,2,[WEP]
1,1,1,1,0,37561812,126935396,60,20110308113017TUE,IDLab,00:kg:9f:94:da:bc,-85.000000,0.000000,2,[WEP]
2,1,2,2,0,37561812,126935396,60,20110308113017TUE,csp,00:6w:3f:53:rp:3a,-84.000000,0.000000,2,[WPA-PSK-TKIP]
3,1,3,3,0,37561812,126935396,60,20110308113017TUE,turbulence,00:kg:9f:6e:wr:1e,-81.000000,0.000000,2,[WPA-PSK-TKIP]
4,1,4,4,0,37561812,126935396,60,20110308113017TUE,李쏀썕 �댁쓽 �ㅽ듃�뚰겕,00:bl:bb:79:2k:8b,-82.000000,0.000000,2,[WPA2-PSK-CCMP]
5,1,5,5,1,37561970,126935365,75,20110308113601TUE,ESLab,00:kg:9f:40:2c:79,-85.000000,0.000000,2,[WEP]
6,1,6,6,1,37561970,126935365,75,20110308113601TUE,QOOKnSHOW,00:9f:a6:a0:hq:95,-85.000000,0.000000,2,[WPA-EAP-TKIP+CCMP][WPA2-EAP-TKIP+CCMP]
7,1,7,7,1,37561970,126935365,75,20110308113601TUE,Yonsei,00:7p:6c:06:kx:01,-84.000000,0.000000,2,[WPA2-EAP-CCMP]
8,1,8,8,1,37561970,126935365,75,20110308113601TUE,archihistory,00:kg:9f:4a:9y:44,-85.000000,0.000000,2,[WEP]
9,1,9,9,1,37561970,126935365,75,20110308113601TUE,myLGNet,00:f4:5a:43:ky:0a,-84.000000,0.000000,2,[WEP]


In [0]:
conn = db_connect(db)
data.to_sql(name='logs',con=conn)
db_close(conn)

In [0]:
query = f"""
CREATE TABLE IF NOT EXISTS logs (
id INTEGER PRIMARY KEY,
user INTEGER NOT NULL,
ssid TEXT NOT NULL,
bssid TEXT NOT NULL,
location INTEGER NOT NULL,
latitude INTEGER NOT NULL,
longitude INTEGER NOT NULL,
);
"""
db_create(db,query)